In [1]:
import numpy as np
from wingstructure.structure import beammechanics as beam
%matplotlib ipympl

In [2]:
# creating geometry and material data

geometry_section = np.load('contour.npy')

youngsmoduli_left = np.ones(len(geometry_section[0])-1)*12.5*10**9
youngsmoduli_center = np.ones(len(geometry_section[1])-1)*12.5*10*9
youngsmoduli_right = np.ones(len(geometry_section[2])-1)*12.5*10**9

shearmoduli_left = np.ones(len(geometry_section[0])-1)*19.6*10**9
shearmoduli_center = np.ones(len(geometry_section[1])-1)*19.6*10**9
shearmoduli_right = np.ones(len(geometry_section[2])-1)*19.6*10**9

thickness_left = np.ones(len(geometry_section[0])-1)*0.003
thickness_center = np.ones(len(geometry_section[1])-1)*0.003
thickness_right = np.ones(len(geometry_section[2])-1)*0.003

youngsmoduli_section = youngsmoduli_left, youngsmoduli_center, youngsmoduli_right
shearmoduli_section = shearmoduli_left, shearmoduli_center, shearmoduli_right
thickness_section = thickness_left, thickness_center, thickness_right

testsection = beam.Crosssection(geometry_section, youngsmoduli_section, shearmoduli_section, thickness_section)

In [3]:
Q = [0,0.03] # testshearforce
T = 0.03 # testmoment

# transform shearforce into principal axis
Q_new = beam.transform(np.array((Q)), testsection.Θ)

In [4]:
# calculate shearflow

shearflow = testsection.shearflow_total(Q_new, T)
shearflow = np.array(shearflow)

In [5]:
# plot shearflow around the profile

from matplotlib import pyplot as plt
plt.figure()
shearflow_plot = testsection.plot_around_profil(shearflow)

shearflow_plot_left = shearflow_plot[0]
shearflow_plot_center = shearflow_plot[1]
shearflow_plot_right = shearflow_plot[2]

plt.plot(shearflow_plot_left[:,0], shearflow_plot_left[:,1], 'orange')
plt.plot(shearflow_plot_center[:,0], shearflow_plot_center[:,1], 'orange')
plt.plot(shearflow_plot_right[:,0], shearflow_plot_right[:,1], 'orange')


center = testsection.shearcenter()
c4 = testsection.c4_point()

geol = testsection.geometries[0]
geoc = testsection.geometries[1]
geor = testsection.geometries[2]

plt.plot(geol[:,0], geol[:,1])
plt.plot(geoc[:,0], geoc[:,1])
plt.plot(geor[:,0], geor[:,1])
plt.plot(center[0], center[1], 'o')
plt.plot(c4[0], c4[1], 'x')
plt.axis('equal')
center

line1 = np.array((shearflow_plot_left[0], shearflow_plot_right[0]))
plt.plot(line1[:,0], line1[:,1], 'orange')

line2 = np.array((shearflow_plot_left[-1], shearflow_plot_right[-1]))
plt.plot(line2[:,0], line2[:,1], 'orange')

FigureCanvasNbAgg()

In [6]:
# plot variable shearflow around profile (for demonstration)

def plotdistri(Qy, Qz, T):
    
    Q = [Qy,Qz]
    Q_new = beam.transform(np.array((Q)), testsection.Θ)
    shearflow = testsection.shearflow_total(Q_new, T)
    shearflow = np.array(shearflow)/30
    
    plt.clf()
    
    shearflow_plot = testsection.plot_around_profil(shearflow)

    shearflow_plot_left = shearflow_plot[0]
    shearflow_plot_center = shearflow_plot[1]
    shearflow_plot_right = shearflow_plot[2]
    
    center = testsection.shearcenter()
    c4 = testsection.c4_point()
    
    plt.plot(geol[:,0], geol[:,1])
    plt.plot(geoc[:,0], geoc[:,1])
    plt.plot(geor[:,0], geor[:,1])
    plt.plot(center[0], center[1], 'o')
    plt.plot(c4[0], c4[1], 'x')
    plt.axis('equal')
    
    
    plt.plot(shearflow_plot_left[:,0], shearflow_plot_left[:,1], 'orange')
    plt.plot(shearflow_plot_center[:,0], shearflow_plot_center[:,1], 'orange')
    plt.plot(shearflow_plot_right[:,0], shearflow_plot_right[:,1], 'orange')

    plt.arrow(c4[0], c4[1], 0, -Qz/10, linewidth=2, color='red')
    plt.arrow(c4[0], c4[1], -Qy/10, 0, linewidth=2, color='red')



    line1 = np.array((shearflow_plot_left[0], shearflow_plot_right[0]))
    plt.plot(line1[:,0], line1[:,1], 'orange')

    line2 = np.array((shearflow_plot_left[-1], shearflow_plot_right[-1]))
    plt.plot(line2[:,0], line2[:,1], 'orange')
    

from ipywidgets import interact, interactive, fixed, interact_manual, FloatSlider
import ipywidgets as widgets

plt.figure()


Q_y_slider = FloatSlider(value=0.0, min=-5, max=5)
Q_z_slider = FloatSlider(value=0.0, min=-5, max=5)
T_slider = FloatSlider(value=0.0, min=-5, max=5)

interact(plotdistri, Qy=Q_y_slider, Qz=Q_z_slider, T=T_slider)

FigureCanvasNbAgg()

interactive(children=(FloatSlider(value=0.0, description='Qy', max=5.0, min=-5.0), FloatSlider(value=0.0, desc…

<function __main__.plotdistri(Qy, Qz, T)>

In [7]:
# calculate twist of the section

testsection.twist(Q_new, T)

2.261141634187205e-08

In [8]:
# calculate deformation vector of the section

N = 1
M_y = 1
M_z = 1
testforces = ((N, M_y, M_z))

testsection.deformation_vec(testforces)

array([1.31806274e-08, 3.30342124e-06, 1.59476867e-07])

In [9]:
# calculate longitudinal deformation in every element of the section

def_element = testsection.deformation_element(testforces)

In [14]:
# calculate longitudinal stress in every element of the section

stress_element = testsection.stress_element(testforces)